In [150]:
import numpy as np
from hmmlearn import hmm
import pandas as pd
import numpy as np
from hmmlearn.base import ConvergenceMonitor
import pickle

In [127]:
def Normalization(df):
    df.iloc[:,2:]= df.iloc[:,2:].apply(lambda x: ((x-x.mean()) / (x.std())))
    return df

In [128]:
def preprocessing(path):
#    gamma = 0.9
    df = pd.read_csv(path, header=None, delimiter=' ')
    
    #Normalize the data
    df = Normalization(df)
    
    #Drop the columns which has all values as Nan
    df.dropna(axis=1, how='all', inplace=True)
    
#     #Get Rewards for each time step : 0 except last time step where reward is -100
#     df['Counter'] = df.index
#     lastRowIndex = df.groupby(0).last().Counter.tolist()
#     df['reward'] = df['Counter'].apply(lambda x : -100 if x in lastRowIndex else 0 )
#     df.drop(columns=['Counter'],inplace=True)
    
    #Rename columns
    df.rename(columns={0: "machine", 1: "time"}, inplace=True)
    
#     #Calculate Monte Carlo Value for each row
#     df1 = df.groupby('machine').last()[['time']].reset_index()
#     df = pd.merge(df, df1, on = 'machine', how = 'left').rename(columns ={'time_x':'time','time_y':'lastTimeStamp'})
#     #df['MC_Val'] = (gamma ** (df['lastTimeStamp'] - df['time'] )) * (-100)
#     df['MC_Val'] = (df['lastTimeStamp'] - df['time'] )
#     df = df.drop(columns='lastTimeStamp')
    
    return df

In [129]:
df = preprocessing("../CMAPSSData/train_FD001.txt")

In [130]:
df.head()

,machine,time,2,3,5,6,7,8,9,10,...,14,15,16,17,18,19,20,21,24,25
0,1,1,-0.315972,-1.372920,0.999976,-1.721684,-0.134252,-0.925914,0.999976,0.14168,...,0.999976,-0.266460,0.334254,-1.058865,-0.269065,-0.603801,0.999976,-0.781691,1.348460,1.194398
1,1,2,0.872701,-1.031695,0.999976,-1.061754,0.211523,-0.643710,0.999976,0.14168,...,0.999976,-0.191578,1.174871,-0.363637,-0.642829,-0.275845,0.999976,-0.781691,1.016503,1.236892
2,1,3,-1.961827,1.015652,0.999976,-0.661797,-0.413156,-0.525940,0.999976,0.14168,...,0.999976,-1.015279,1.364688,-0.919819,-0.551616,-0.649128,0.999976,-2.073044,0.739873,0.503411
3,1,4,0.324083,-0.008022,0.999976,-0.661797,-1.261284,-0.784812,0.999976,0.14168,...,0.999976,-1.539452,1.961255,-0.224592,-0.520163,-1.971617,0.999976,-0.781691,0.352590,0.777773
4,1,5,-0.864590,-0.690471,0.999976,-0.621801,-1.251498,-0.301511,0.999976,0.14168,...,0.999976,-0.977838,1.052846,-0.780774,-0.521736,-0.339837,0.999976,-0.136014,0.463242,1.059526


In [131]:
X = df.iloc[:,2:]

In [132]:
X

,2,3,5,6,7,8,9,10,11,12,...,14,15,16,17,18,19,20,21,24,25
0,-0.315972,-1.372920,0.999976,-1.721684,-0.134252,-0.925914,0.999976,0.14168,1.121113,-0.516326,...,0.999976,-0.266460,0.334254,-1.058865,-0.269065,-0.603801,0.999976,-0.781691,1.348460,1.194398
1,0.872701,-1.031695,0.999976,-1.061754,0.211523,-0.643710,0.999976,0.14168,0.431920,-0.798074,...,0.999976,-0.191578,1.174871,-0.363637,-0.642829,-0.275845,0.999976,-0.781691,1.016503,1.236892
2,-1.961827,1.015652,0.999976,-0.661797,-0.413156,-0.525940,0.999976,0.14168,1.008131,-0.234578,...,0.999976,-1.015279,1.364688,-0.919819,-0.551616,-0.649128,0.999976,-2.073044,0.739873,0.503411
3,0.324083,-0.008022,0.999976,-0.661797,-1.261284,-0.784812,0.999976,0.14168,1.222798,0.188044,...,0.999976,-1.539452,1.961255,-0.224592,-0.520163,-1.971617,0.999976,-0.781691,0.352590,0.777773
4,-0.864590,-0.690471,0.999976,-0.621801,-1.251498,-0.301511,0.999976,0.14168,0.714376,-0.516326,...,0.999976,-0.977838,1.052846,-0.780774,-0.521736,-0.339837,0.999976,-0.136014,0.463242,1.059526
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20626,-0.178817,-1.031695,0.999976,1.617961,1.216229,2.188322,0.999976,0.14168,-2.189276,1.315035,...,0.999976,1.979996,-2.607906,2.278227,-0.322534,1.425260,0.999976,2.446692,-1.805129,-2.921042
20627,-0.727436,-1.714145,0.999976,1.717950,2.279651,2.738285,0.999976,0.14168,-2.833277,1.878530,...,0.999976,1.867673,-2.350298,1.722045,-0.380198,1.913194,0.999976,1.155339,-2.856326,-1.203735
20628,0.186928,-0.008022,0.999976,1.477975,1.946924,2.138325,0.999976,0.14168,-2.742891,2.019404,...,0.999976,2.054877,-1.902872,2.000136,-0.141680,3.265013,0.999976,3.092369,-2.081760,-3.292402
20629,-0.498845,1.015652,0.999976,1.098016,2.403608,1.955004,0.999976,0.14168,-3.036645,2.160278,...,0.999976,3.178105,-2.363856,1.861090,-0.233942,2.579771,0.999976,1.155339,-2.911652,-2.085021


In [133]:
lengths = df.groupby("machine").size().tolist()

## We have sequence of trajectories (observations) and their lengths. Now we can use Expectation Maximization to estimate the parameters for the HMM

In [135]:
#Character codes: ‘s’ for startprob, ‘t’ for transmat, ‘m’ for means and ‘c’ for covars.

In [170]:
len(X)

20631

In [171]:
sum(lengths)

20631

In [187]:
lr = hmm.GaussianHMM(n_components=4, covariance_type="diag",init_params="cm", params="mt", random_state=42)
lr.startprob_ = np.array([1.0, 0.0, 0.0, 0.0])
lr.transmat_ = np.array([[0.5, 0.5, 0.0, 0.0],[0.0, 0.5, 0.5, 0.0],[0.0, 0.0, 0.5, 0.5],[0.0,0.0,0.0,1.0]])
lr.fit(X, lengths=lengths)
#lr.fit(X[:193])

GaussianHMM(init_params='cm', n_components=4, params='mt', random_state=42)

In [317]:
with open("HMM.pkl", "wb") as file: pickle.dump(lr, file)

In [318]:
lr_copy = pickle.load( open( "HMM.pkl", "rb" ) ) #Just shown here to document how to load the HMM model.

In [188]:
lr.transmat_

array([[0.98789878, 0.01210122, 0.        , 0.        ],
       [0.        , 0.98815232, 0.01184768, 0.        ],
       [0.        , 0.        , 0.96299289, 0.03700711],
       [0.        , 0.        , 0.        , 1.        ]])

In [190]:
lr.predict(X[194+lengths[1]+1: 194+lengths[1]+2])

array([0])

In [193]:
lr.covars_.shape

(4, 21, 21)

In [194]:
lr.predict_proba(X[:193])

array([[1.00000000e+000, 0.00000000e+000, 0.00000000e+000,
        0.00000000e+000],
       [1.00000000e+000, 5.24975360e-168, 0.00000000e+000,
        0.00000000e+000],
       [1.00000000e+000, 1.80610877e-166, 0.00000000e+000,
        0.00000000e+000],
       [1.00000000e+000, 5.11487070e-164, 0.00000000e+000,
        0.00000000e+000],
       [1.00000000e+000, 4.51016914e-161, 0.00000000e+000,
        0.00000000e+000],
       [1.00000000e+000, 2.57065866e-159, 0.00000000e+000,
        0.00000000e+000],
       [1.00000000e+000, 7.09508866e-156, 0.00000000e+000,
        0.00000000e+000],
       [1.00000000e+000, 5.34652329e-153, 0.00000000e+000,
        0.00000000e+000],
       [1.00000000e+000, 4.28928415e-150, 0.00000000e+000,
        0.00000000e+000],
       [1.00000000e+000, 2.04051681e-147, 0.00000000e+000,
        0.00000000e+000],
       [1.00000000e+000, 3.73981271e-145, 0.00000000e+000,
        0.00000000e+000],
       [1.00000000e+000, 3.41359198e-142, 0.00000000e+000,
      

In [196]:
#Now we need to calculate reward for each state estimate using the formula
#Since there is no action: reward = c1 * (4-st), c1=50

#Lets calculate the reward for each state
#Reward for state1 = 50 * (4-1)= 150
#Reward for state2 = 50 * (4-2)= 100
#Reward for state3 = 50 * (4-3)= 50
#Reward for state4 = 50 * (4-4)= 0

In [206]:
dict_rewards = {0:150, 1:100, 2:50, 3:0} #Note that we are using the state index to denote the state. Hence, 0 to 4

In [198]:
#We have sequence of predicted probabilities using the lr.predict_proba. For each step we will calculate the reward using
#the weighted probability of dict_rewards

In [271]:
def cal_reward(states_prob, dict_rewards):
    reward = 0
    for i,prob in enumerate(states_prob):
        reward+= prob * dict_rewards[i]
    #print(reward)
    return reward

In [272]:
rewards = []

In [273]:
start=0
for length in lengths:
    #print(start,start+length, length)
    for states_prob in lr.predict_proba(X[start:start+length]):
        rewards.append(cal_reward(states_prob, dict_rewards))
    start += length

In [290]:
returns=[]
start=0
for length in lengths:
    #print(start, start+length)
    reward_seq = rewards[start:start+length]
    #print(reward_seq)
    #print("##################################")
    for i in range(len(reward_seq)):
        returns.append(sum(reward_seq[i:]))
    start+=length
        

In [291]:
len(returns)

20631

In [297]:
df["Return"] = np.array(returns) #We will use this as our dataframe for Predictron

In [304]:
df.to_csv("HMM_data.csv")

In [300]:
df[193:194]

,machine,time,2,3,5,6,7,8,9,10,...,15,16,17,18,19,20,21,24,25,Return
193,2,2,1.969937,-1.031695,0.999976,-1.721684,-0.566471,-1.755858,0.999976,0.14168,...,-1.127601,1.744321,-1.615046,-0.663798,-0.678458,0.999976,-0.781691,1.34846,1.097401,35793.099389


In [309]:
def checkSequence(seq_num, lengths):
    start=0
    for i in range(len(lengths)):
        if(i+1 == seq_num):
            return lr.predict_proba(X[start:start+length])
        start+=length

In [310]:
checkSequence(5, lengths)

array([[1.00000000e+000, 0.00000000e+000, 0.00000000e+000,
        0.00000000e+000],
       [1.00000000e+000, 0.00000000e+000, 0.00000000e+000,
        0.00000000e+000],
       [1.00000000e+000, 0.00000000e+000, 0.00000000e+000,
        0.00000000e+000],
       [1.00000000e+000, 0.00000000e+000, 0.00000000e+000,
        0.00000000e+000],
       [1.00000000e+000, 0.00000000e+000, 0.00000000e+000,
        0.00000000e+000],
       [1.00000000e+000, 0.00000000e+000, 0.00000000e+000,
        0.00000000e+000],
       [1.00000000e+000, 0.00000000e+000, 0.00000000e+000,
        0.00000000e+000],
       [1.00000000e+000, 0.00000000e+000, 0.00000000e+000,
        0.00000000e+000],
       [1.00000000e+000, 0.00000000e+000, 0.00000000e+000,
        0.00000000e+000],
       [1.00000000e+000, 0.00000000e+000, 0.00000000e+000,
        0.00000000e+000],
       [1.00000000e+000, 0.00000000e+000, 0.00000000e+000,
        0.00000000e+000],
       [1.00000000e+000, 0.00000000e+000, 0.00000000e+000,
      

In [315]:
lr.predict(X[192:192+287])

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3])

In [314]:
lengths

[192,
 287,
 179,
 189,
 269,
 188,
 259,
 150,
 201,
 222,
 240,
 170,
 163,
 180,
 207,
 209,
 276,
 195,
 158,
 234,
 195,
 202,
 168,
 147,
 230,
 199,
 156,
 165,
 163,
 194,
 234,
 191,
 200,
 195,
 181,
 158,
 170,
 194,
 128,
 188,
 216,
 196,
 207,
 192,
 158,
 256,
 214,
 231,
 215,
 198,
 213,
 213,
 195,
 257,
 193,
 275,
 137,
 147,
 231,
 172,
 185,
 180,
 174,
 283,
 153,
 202,
 313,
 199,
 362,
 137,
 208,
 213,
 213,
 166,
 229,
 210,
 154,
 231,
 199,
 185,
 240,
 214,
 293,
 267,
 188,
 278,
 178,
 213,
 217,
 154,
 135,
 341,
 155,
 258,
 283,
 336,
 202,
 156,
 185,
 200]